In [13]:
print("Hello WOrld!!!")

Hello WOrld!!!


In [14]:
# ! pip install numpy
# ! pip install pandas
# ! pip install torch

# ! pip install scikit-learn

In [15]:
import numpy as np
import torch
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [16]:
df  = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

df.shape

(569, 33)

In [17]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [18]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace= True)

In [19]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [20]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2, random_state=42)

In [21]:
print(X_train.shape)
print(df.shape)

(455, 30)
(569, 31)


In [22]:
scaler = StandardScaler()


X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
encoder = LabelEncoder()

y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

Numpy Arrays to PyTorch Tensor

In [24]:
X_train_tensor = torch.from_numpy(X_train_scaled)
X_test_tensor = torch.from_numpy(X_test_scaled)
y_train_tensor = torch.from_numpy(y_train_encoded)
y_test_tensor = torch.from_numpy(y_test_encoded)

In [25]:
X_train_tensor

tensor([[-1.4408, -0.4353, -1.3621,  ...,  0.9320,  2.0972,  1.8865],
        [ 1.9741,  1.7330,  2.0917,  ...,  2.6989,  1.8912,  2.4978],
        [-1.4000, -1.2496, -1.3452,  ..., -0.9702,  0.5976,  0.0579],
        ...,
        [ 0.0488, -0.5550, -0.0651,  ..., -1.2390, -0.7086, -1.2715],
        [-0.0390,  0.1021, -0.0314,  ...,  1.0500,  0.4343,  1.2134],
        [-0.5486,  0.3133, -0.6035,  ..., -0.6110, -0.3345, -0.8463]],
       dtype=torch.float64)

Creating the Model

In [26]:
class SimpleNN():
    
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)
        
    def forward_pass(self, X):
        y = torch.matmul(X, self.weights) + self.bias
        
        y_pred = torch.sigmoid(y)
        return y_pred
        
        
    def loss_calculation(self, y_pred, y_true):
        epsilon = 1e-7
        
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
        
        loss = -(y_true*torch.log(y_pred) + (1-y_true)*torch.log(1-y_pred)).mean()
        
        return loss
    
 
        
    
        
        

Parameters

In [27]:
learning_rate = 0.05
epochs = 500

Training Pipeline

In [28]:
# creating model
model = SimpleNN(X_train_tensor)

print(model.bias)

for epoch in range(epochs):
    
    # forward pass
    
    y_pred = model.forward_pass(X_train_tensor)

    # calculate loss
    loss = model.loss_calculation(y_pred, y_train_tensor)
    print(f" for epoch : {epoch}/{epochs}, loss is {loss.item()}")
    
    # backward pass
    
    loss.backward()


    # parameters update
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad

    # zero gradientsy_train_tensor
    model.weights.grad.zero_()
    model.bias.grad.zero_()
    

tensor([0.], dtype=torch.float64, requires_grad=True)
 for epoch : 0/500, loss is 3.6124326274773275
 for epoch : 1/500, loss is 3.5481924334424
 for epoch : 2/500, loss is 3.481123478940542
 for epoch : 3/500, loss is 3.413144508125074
 for epoch : 4/500, loss is 3.3445658976395145
 for epoch : 5/500, loss is 3.2747331597003995
 for epoch : 6/500, loss is 3.20409797148116
 for epoch : 7/500, loss is 3.132289500569909
 for epoch : 8/500, loss is 3.0586889685041334
 for epoch : 9/500, loss is 2.9840201261481867
 for epoch : 10/500, loss is 2.907285372582708
 for epoch : 11/500, loss is 2.8293071056791237
 for epoch : 12/500, loss is 2.748155963581068
 for epoch : 13/500, loss is 2.6677690838747163
 for epoch : 14/500, loss is 2.5876000524215184
 for epoch : 15/500, loss is 2.507362485939635
 for epoch : 16/500, loss is 2.4281085883886417
 for epoch : 17/500, loss is 2.3488000400776885
 for epoch : 18/500, loss is 2.267195003865613
 for epoch : 19/500, loss is 2.1860416232556377
 for epo

In [30]:
with torch.no_grad():
   #oss is 0.9898135662078857
 #for epoch : 3/500, loss is 0.989813566207885
   y_test_pred = model.forward_pass(X_test_tensor)
   y_test_pred_labels = (y_test_pred >= 0.5).float()
   
   accuracy = (y_test_pred_labels.squeeze() == y_test_tensor).float().mean()
   
   print(f"Test Accuracy: {accuracy.item()*100:.2f}%")

Test Accuracy: 62.28%


# Using Torch's NN Module 

In [31]:
import torch.nn as nn

In [32]:
class Model(nn.Module):
    
    def __init__(self, num_features, hidden_layer1_size):
        super().__init__()
        
        
        self.num_features = num_features
        self.hidden_layer1_size = hidden_layer1_size
        
        self.neural_network = nn.Sequential(
            nn.Linear(num_features, hidden_layer1_size),
            nn.ReLU(),
            nn.Linear(hidden_layer1_size, 1),
            #nn.Sigmoid()
        )
        
        
        
    def forward(self, features):
       
        activation2_out = self.neural_network(features)

        return activation2_out

In [33]:
print(X_train_tensor.shape)

torch.Size([455, 30])


In [38]:

hidden_layer_size = 15
learning_rate = 0.05
epochs = 5000

#loss_function = nn.BCELoss() # binary cross entropy loss
loss_function = nn.BCEWithLogitsLoss()

model = Model(X_train_tensor.shape[1], hidden_layer_size)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
# model

    # internally NN Module calls forward method
    y_pred = model(X_train_tensor.float()) # same as model.forward(features)
    loss = loss_function(y_pred, y_train_tensor.float().unsqueeze(1))

    
    optimizer.zero_grad()
    
    loss.backward()
    optimizer.step()
    #print(f" for epoch : {epoch}/{epochs}, loss is {loss.item()}")


y_pred_f = model(X_test_tensor.float())

y_pred_f = (y_pred_f >= 0.5).float()

accuracy = (y_pred_f == y_test_tensor).float().mean()

print(accuracy.item())


    # print(y_pred)
# 0.5323176383972168

# 0.5280086398124695 - ADAM
# 0.5344721674919128 - SGD

0.5323176383972168


In [39]:
y_pred = model(X_test_tensor.float())

y_pred = (y_pred >= 0.5).float()

accuracy = (y_pred == y_test_tensor).float().mean()

print(accuracy.item())

0.5323176383972168
